# Abflussmodellierung mittels Potenzfunktion
Dieses Notebook basiert auf einer nichtlinearen Regression der Form: 
\[ Q = a \cdot (h - h_0)^b \]
zur Modellierung des Abflusses in Abhängigkeit vom hydraulischen Potential.

Erstellt am: 2025-05-15

In [ ]:
# 📦 Installiere erforderliche Pakete (nur beim ersten Mal nötig)
# Entferne die Kommentarzeichen, wenn du in Google Colab arbeitest
# install.packages('terra')
# install.packages('mgcv')
# install.packages('dplyr')

library(terra)
library(mgcv)
library(dplyr)

In [ ]:
library(terra)
library(mgcv)
library(dplyr)

## 📥 Einlesen der PQ-Daten

In [ ]:
df_pq <- read.csv("./abstrom_aggregiert_subdomain_durchfluss.dat", sep=";")

plot(df_pq$h, df_pq$Q, pch = 19, col = "blue",
     main = "Nichtlineare Regression: Q ~ a*(h - h0)^b", xlab = "FINIT_mean (h)", ylab = "Q")
print('✅')

## 📈 Schätzung Startwerte via Log-Log

In [ ]:
h0_start <- min(df_pq$h, na.rm = TRUE) - 0.01
df_valid <- subset(df_pq, h > (h0_start + 0.01) & Q > 0)
log_model <- lm(log(Q) ~ log(h - h0_start), data = df_valid)
b_start <- as.numeric(coef(log_model)[2])
a_start <- exp(as.numeric(coef(log_model)[1]))

## 🔁 Nichtlineare Regression

In [ ]:
model_nls <- nls(Q ~ a * (h - h0)^b,
                 data = df_valid,
                 start = list(a = a_start, h0 = h0_start, b = b_start),
                 algorithm = "port",
                 lower = c(a = 0, h0 = h0_start - 0.5, b = 0.1),
                 upper = c(a = 1e6, h0 = h0_start + 0.5, b = 5),
                 control = list(maxiter = 1000, warnOnly = TRUE))
summary(model_nls)

In [ ]:
preds <- predict(model_nls)
rmse <- sqrt(mean((df_pq$Q - preds)^2))
plot(df_pq$h, df_pq$Q, pch = 19, col = "blue",
     main = "Nichtlineare Regression: Q ~ a*(h - h0)^b", xlab = "FINIT_mean (h)", ylab = "Q")
lines(sort(df_valid$h), predict(model_nls, newdata = data.frame(h = sort(df_valid$h))), col = "red", lwd = 2)
print('✅')

## 📌 Koeffizienten und Bereichsauswertung

In [ ]:
coef(model_nls)
a <- coef(model_nls)[["a"]]
h0 <- coef(model_nls)[["h0"]]
b <- coef(model_nls)[["b"]]

range_head = c(191, 193)
q_result_min <- a*( range_head[1]- h0 )^b
q_result_max <- a*( range_head[2]- h0 )^b

area = 427960
q_result_min / area
q_result_max / area

## 📦 Verarbeitung der Shapefiles

In [ ]:
abstrom <- terra::vect("./V11_Abstrom_Nodes_Hyhead_Darcy_Flux_Rate_budget.shp", crs = "+init=epsg:31467")
abstrom <- data.frame(abstrom)
aggregiert_df <- abstrom %>%
  group_by(TIME) %>%
  summarise(
    FINIT_mean = mean(FINIT, na.rm = TRUE),
    BDRFLOW_sum = -sum(BDRFLOW, na.rm = TRUE),
    VINIT_mean = mean(VINIT, na.rm = TRUE)
  )

In [ ]:
abstrom_result <- terra::vect("./V11_Abstrom_Nodes_Hyhead_Darcy_Flux_Rate_budget_period_expression_editor.shp", crs = "+init=epsg:31467")
abstrom_result <- data.frame(abstrom_result)
aggregiert_df_result <- abstrom_result %>%
  group_by(TIME) %>%
  summarise(
    FINIT_mean = mean(FINIT, na.rm = TRUE),
    BDRFLOW_sum = -sum(BDRFLOW, na.rm = TRUE),
    VINIT_mean = mean(VINIT, na.rm = TRUE)
  )

## 📊 Zeitliche Entwicklung

In [ ]:
# Hydraulic head 
plot(aggregiert_df$TIME, aggregiert_df$FINIT_mean, type="l", col="purple", main = "Hyhead") 
lines(aggregiert_df_result$TIME, aggregiert_df_result$FINIT_mean, col="cyan")

# Geschwindigkeit 
plot(aggregiert_df$TIME, aggregiert_df$VINIT_mean, type="l", col="purple", main = "Geschwindigkeit") 
lines(aggregiert_df_result$TIME, aggregiert_df_result$VINIT_mean, col="cyan")

# Abfluss 
plot(aggregiert_df$TIME, aggregiert_df$BDRFLOW_sum, type="l", col="purple", main = "Abfluss") 
lines(aggregiert_df_result$TIME, aggregiert_df_result$BDRFLOW_sum, col="cyan")

# Referenzlinien
abline(h = q_result_min, col="red")
abline(h = q_result_max, col="red")
print('✅')